## Baixando mangás com Python

Alvo: centraldemangas.org
Mangá Inicial: One Piece
Ferramentas que vamos usar:

- editor de texto (sublime text3)
- terminal
- Python3
    - argparse
    - os
    - time
    - urllib.request
    - BeautifulSoup
    
### Porque fazer isso?
Bom, primeiro porque parece ser legal :)
Segundo porque essa que parece ser uma tarefa simples, pode acabar se tornando um pouco complexa, envolvendo alguns conceitos que podem ser desconhecidos numa primeira vez, mas que trazem um grande aprendizado.

O workflow é muito simples: vamos dizer para o programa qual o capítulo queremos baixar.
Agora vamos falar um pouco, mas bem rápido sobre as ferramentas que vamos usar.

### Standard libraries
- argparse
    - O módulo argparse é usado para criar CLIs de maneira fácil e amigável.
- os
    - Esse módulo fornece uma maneira portável de usar funcionalidade do sistema operacional.
- time
    - Funções realacionadas ao tempo.
- urllib.request
    - Esse módulo define funções e classes que ajudam a "abrir" e "ler" URLs (maioria HTTP).
- BeautifulSoup
    - É uma biblioteca escrita em Python projetada para parsear páginas.

Bom, vamos deixar de papo e começar a programWAIT! Antes disso precisamos fazer uma coisa.

### Preparar o ambiente
Iremos isolar nosso ambiente do resto do sistema, para podermos instalar todas as dependências do projeto de maneira segura.
````shell
$ mkvirtualenv -p python3 pymanga
$ pip install beautifulsoup4
````

Preparado nosso ambiente, e instalada nossas dependências, agora sim vamos iniciar a programação :)

### 1ª etapa - Função `main()`
````python
import argparse
import os
import time
import urllib.request

from bs4 import BeautifulSoup


def main():
    parser = argparse.ArgumentParser(
        prog='PyManga',
        description='An easily way to download your favorite mangas'
    )
    parser.add_argument(
        '-m', '--manga',
        type=str, help="Type the manga's name"
    )
    parser.add_argument(
        '-c', '--chapter',
        type=str, help='Type the chapter you wish'
    )

    args = parser.parse_args()

    if args.manga and args.chapter:
        download_chapter(args.chapter)
````

### 2ª etapa - Função `create_folder()`

````python
def create_folder(chapter):
    folder = 'capitulo-{}'.format(chapter)
    try:
        os.mkdir(folder)
        return folder
    except OSError:
        print('Diretório já existente')
        return folder
````

### 3ª etapa - Função `download_chapter()`
````python
BASE_URL_ONE_PIECE = 'http://mangaop.info/capitulos/{}#1'

def download_chapter(chapter):
    url = BASE_URL_ONE_PIECE.format(chapter)
    req = urllib.request.Request(url)
    content = urllib.request.urlopen(req).read()
````
O que nós fizemos até o momento, foi ajustar a nossa URL com o capítulo do mangá que o usuário quer baixar, usando o método `format()` da classe string. 
Depois disso, nós criamos um objeto `Request` usando a url como parâmetro. Após isso, usamos este objeto como parâmetro para a função `urlopen()` que de fato abre uma conexão e captura a página. O método `read()` retorna conteúdo deste objeto.

````python
    soup = BeautifulSoup(content, 'html.parser')
````
Tendo o html da página, agora nós criamos um objeto BeautifulSoup, usando o `html.parser` como parser. Agora, antes de seguir para o passo seguinte, vale dar uma olhada no html. Primeiramente nota-se que o HTML gerado no navegador é **muito** diferente do HTML que a requisição retorna. Um dos motivos é que essa requisição feita pelo Python, não tem como executar os códigos JS contidos na páginas, que depois de carregados podem e alteram o DOM da página. 

Sendo assim, tive que analisar o HTML para ver como faria para capturar o que eu quero. Após finalizar esta análise, pude notar que a URL da imagem do capítulo estava dentro de uma variável de um código JS, ou seja, dentro de uma tag `<script>`. Bom, aqui veio o pulo do gato:
````python
    words = soup.findAll('script')[16].string.split()
````
O método `findAll()` me retorna uma lista com todas as ocorrências da tag que eu passar como parâmetro. Após isso eu notei que a script que estava no índice 16 era o que continha a variável com a URL que eu queria. Então uso o atributo `string` que me retorna o toda a tag, inclusive seu conteúdo, como texto, e dei um `split()`, que retorna uma lista com as palavras naquela string, usando como separador o espaço entre as palavras.
````python
manga_url = ''
    for word in words:
        # check if word contains 'http' string
        if 'http' in word:
            # if so, assign it to manga_url and break the for
            manga_url = word
            break
````